In [10]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
import requests

import warnings
warnings.filterwarnings('ignore')

import os
import selenium
from selenium import webdriver
import re
from datetime import datetime as dt, timedelta
import datetime
from tqdm import tqdm

import pandas as pd
import time
import math
import numpy as np

from dateutil.parser import parse
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.keys import Keys
import lxml
import lxml.html

In [2]:
comp_list = pd.read_excel('./data/comp_list.xlsx').iloc[:,1:]
comp_list

,서비스업,제조/화학,의료/제약/복지,유통/무역/운송,교육업,건설업,IT/웹/통신,미디어/디자인,은행/금융업,기관/협회
0,스타벅스커피코리아,삼성전자,한미약품,씨제이올리브영,해커스 교육그룹,한국토지주택공사,쿠팡,씨제이씨지브이,국민건강보험공단,대한무역투자진흥공사
1,한국맥도날드,엘지전자,대웅제약,에프알엘코리아,동화세상에듀코,한전KPS,㈜티몬,롯데시네마,새마을금고,삼일회계법인
2,롯데리아,엘지디스플레이,종근당,한샘,에스티유니타스,현대엔지니어링,위메프,씨제이이앤엠,근로복지공단,한영회계법인
3,하나투어,현대자동차,녹십자,이마트,에듀윌,현대건설,삼성에스디에스,교원,농협은행,안진회계법인
4,유베이스,엘지이노텍,한국유나이티드제약,홈플러스,대교,대우건설,엘지씨엔에스,문화방송,중소기업은행,삼정회계법인
...,...,...,...,...,...,...,...,...,...,...
95,마이크임팩트,메디앙스,펜믹스,올림푸스한국,창의와탐구,평화엔지니어링,데이터스트림즈,아이코닉스,예금보험공사,서울산업진흥원
96,부킹닷컴코리아,한국가스공사,종근당바이오,필립스코리아,하이컨시,비엔에이치,위메이드,엠디마케팅,카카오페이,한국식품연구원
97,한국기업데이터,타이코에이엠피,경희의료원,트라이씨클,글로벌콘텐츠리퍼블릭,세아네트웍스,이카운트,한국학술정보,제이비우리캐피탈,법무법인세종
98,바이오톡스텍,에스케이케미칼,세이브더칠드런코리아,브리티쉬아메리칸토바코코리아,제이엠커리어,경남기업,에스케이브로드밴드,한국우편사업진흥원,엔에이치농협캐피탈,세종문화회관


In [3]:
comp_list_pre = pd.DataFrame(columns=['category','name'])
for i in comp_list.columns:
    t = pd.DataFrame({'category':i, 'name':comp_list[i]})
    comp_list_pre = pd.concat([comp_list_pre, t], axis=0)
    
comp_list_pre = comp_list_pre.reset_index(drop=True)

In [4]:
comp_list_pre

,category,name
0,서비스업,스타벅스커피코리아
1,서비스업,한국맥도날드
2,서비스업,롯데리아
3,서비스업,하나투어
4,서비스업,유베이스
...,...,...
995,기관/협회,서울산업진흥원
996,기관/협회,한국식품연구원
997,기관/협회,법무법인세종
998,기관/협회,세종문화회관


In [47]:
comp_inout_salary = pd.DataFrame(columns=['name','comp_in','comp_out','new_salary','average_salary'])

In [11]:
op = ChromeOptions()
# op.add_argument('--headless')
op.add_argument('--no-sandbox')
op.add_argument('--disable-dev-shm-usage')

browser = Chrome('./chromedriver.exe', options=op)
browser.get('https://www.kreditjob.com/')


search = browser.find_element_by_xpath('//*[@id="root"]/div/div[5]/div[1]/div/div[2]/div/div[2]/div/input')
search.send_keys(name)
search.send_keys(Keys.RETURN)

In [48]:
op = ChromeOptions()
# op.add_argument('--headless')
op.add_argument('--no-sandbox')
op.add_argument('--disable-dev-shm-usage')

browser = Chrome('./chromedriver.exe', options=op)

not_list = []
for name in comp_list_pre['name']:
    try:
        browser.get('https://www.kreditjob.com/')
        search = browser.find_element_by_xpath('//*[@id="root"]/div/div[5]/div[1]/div/div[2]/div/div[2]/div/input')
        search.send_keys(name)
        time.sleep(0.5)
        search.send_keys(Keys.RETURN)
        time.sleep(2)
        try:
            comp_in = browser.find_element_by_css_selector('#root > div > div.body-container > div.company-container > div.company-wrapper > div.company-contents > section.company-info-container > section.company-info-info-container > div:nth-child(3) > div:nth-child(1) > span.company-info-ratio').text
        except:
            comp_in = np.nan
        try:
            comp_out = browser.find_element_by_css_selector('#root > div > div.body-container > div.company-container > div.company-wrapper > div.company-contents > section.company-info-container > section.company-info-info-container > div:nth-child(3) > div.company-info-btn.latter > span.company-info-ratio').text
        except:
            comp_out = np.nan
        new_salary = browser.find_element_by_css_selector('#root > div > div.body-container > div.company-container > div.company-wrapper > div.company-contents > section.company-salary-container > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div > div > div.salary-label-right > span').text
        average_salary = browser.find_element_by_css_selector('#root > div > div.body-container > div.company-container > div.company-wrapper > div.company-contents > section.company-salary-container > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div > div > div.salary-label-right > span').text

        data_to_insert = {'name':name, 'comp_in':comp_in, 'comp_out':comp_out, 'new_salary':new_salary, 'average_salary':average_salary}
        comp_inout_salary = comp_inout_salary.append(data_to_insert, ignore_index=True)
        time.sleep(1)
    except:
        not_list.append(name)
        print(name)
        time.sleep(1)

롯데리아 
트랜스코스모스코리아
에스케이쉴더스
빕스 
에스원
코웨이
던킨/던킨도너츠 
뚜레쥬르 
케이에이
케이텍맨파워
세일즈웍스 코리아
신세계푸드
에스피씨그룹
참좋은여행
코티티시험연구원
티지아이프라이데이스 
키자니아
에프아이티아이시험연구원
커리어케어
인터파크투어
제이더블유메리어트호텔
삼성중공업
삼성에스디아이
배스킨라빈스 
와이지-원
한솔섬유
효성
한국항공우주산업
팬택
신성통상
대웅제약
한국유나이티드제약
대원제약
사회복지법인월드비전
자생한방병원
사회복지법인 밀알복지재단
로킷헬스케어
태준제약
제이더블유중외신약
메드트로닉코리아
펜믹스
세이브더칠드런코리아
한샘
롯데쇼핑 롯데마트사업본부
삼성에버랜드
서브원
제너시스비비큐
한진
씨제이이엔엠 오쇼핑부문
컬리
포스코인터내셔널
삼성물산 상사부문
유한책임회사디캐이
로레알코리아
텐바이텐
엘브이엠에치코스메틱스
비티지
이니스프리
케이제이씨디스플레이코퍼레이션
미스터피자 
클라우드나인마케팅
해커스 교육그룹
동화세상에듀코
에듀플렉스 
비스픽
와이비엠시사닷컴
캐럿글로벌
한국방송통신대학교
케이지패스원
타임컨텐츠
충남대학교
SBS아카데미게임학원
씨앤씨미술교육지원센터
HIENGLISH KOREA
지일이삼공교육
문정아중국어연구소
이지수능교육
계명대학교 산학협력단
에듀라인
인천영어마을
피엠디아카데미
제이엠커리어
한전KPS
현대건설
케이알산업
금호산업
우리피앤에스
현대산업개발
동부건설
서영엔지니어링
대림건설
포스코엔지니어링
제일건설
제이에스비도시환경
성창이엔씨
신안그룹
신세계건설
㈜티몬
SK텔레콤
효성아이티엑스
넥슨코리아
요기요
배달의민족
에스케이
여기어때
에스케이커뮤니케이션즈
에스케이엠앤서비스
메가존
아프리카티비
오픈타이드코리아
지에스아이티엠
신세계아이앤씨
엠서클
인크루트알바콜
브랜디
옐로오투오
롯데시네마 
대학내일
케이피알앤드어소시에이츠
NE능률
와이지엔터테인먼트
부산국제영화제
채널에이
현대퓨처넷
로커스
디자인하우스
지엔앰퍼포먼스
새마을금고
근로복지공단
DB손해보험
푸르덴셜생명보험
에스비아이저축은행
케이비국민카드
케이비증권
웰컴크레디라인대부
광주은행

In [69]:
comp_inout_salary

,회사명,comp_in,comp_out,new_salary,average_salary
0,스타벅스커피코리아(주),45.0,37.0,"1,945 만원","2,729 만원"
1,한국맥도날드(유),25.0,24.0,"2,301 만원","3,394 만원"
2,(주)하나투어,98.0,30.0,"2,457 만원","3,529 만원"
3,(주)유베이스,86.0,88.0,"1,818 만원","2,673 만원"
4,(주)커피빈코리아,66.0,70.0,"1,442 만원","2,101 만원"
...,...,...,...,...,...
843,국토연구원,21.0,22.0,"3,341 만원","8,542 만원"
844,서울산업진흥원,16.0,13.0,"3,301 만원","4,969 만원"
845,한국식품연구원,19.0,18.0,"3,817 만원","7,131 만원"
846,법무법인세종,18.0,18.0,"3,684 만원","5,667 만원"
